#### Importing required libraries

In [1]:
# install packages
install.packages(c("dplyr", "ggplot2", "tidyr", "readr", "purrr", "stringr",
                "lubridate", "data.table", "plyr", "janitor", "reshape2", "readxl"))


The downloaded binary packages are in
	/var/folders/1j/rn9q783j7sjdszqsfc6_89sw0000gn/T//RtmpfLJUTr/downloaded_packages


In [2]:
# load packages
library(dplyr)
library(ggplot2)
library(tidyr)
library(readr)
library(purrr)
library(stringr)
library(lubridate)
library(data.table)
library(plyr)
library(janitor)
library(reshape2)
library(readxl)
library(mice)
library(gridExtra)
library(rlang)


Attaching package: 'dplyr'


The following objects are masked from 'package:stats':

    filter, lag


The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union



Attaching package: 'lubridate'


The following objects are masked from 'package:base':

    date, intersect, setdiff, union


Warning message:
"package 'data.table' was built under R version 4.3.3"

Attaching package: 'data.table'


The following objects are masked from 'package:lubridate':

    hour, isoweek, mday, minute, month, quarter, second, wday, week,
    yday, year


The following object is masked from 'package:purrr':

    transpose


The following objects are masked from 'package:dplyr':

    between, first, last


------------------------------------------------------------------------------

You have loaded plyr after dplyr - this is likely to cause problems.
If you need functions from both plyr and dplyr, please load plyr first, then dplyr:
library(plyr); library(dplyr)

--

In [3]:
# define path we are working in
path <- "/Users/stevenschepanski/Documents/04_ANALYSIS/Kneipp"

In [4]:
# Read in data
sosci <- read_csv(paste0(path, "/data/Sosci.csv"))

Rows: 13360 Columns: 50
-- Column specification --------------------------------------------------------
Delimiter: ","
chr  (3): serial_ID, questionnaire_time_point, kindergarten_ID
dbl (47): zipcode, number_adults, number_minors, age_oldest, age_youngest, f...

i Use `spec()` to retrieve the full column specification for this data.
i Specify the column types or set `show_col_types = FALSE` to quiet this message.


# SOSCI

In [5]:
# Display the first few rows of the DataFrame using the 'head()' method
head(sosci)

serial_ID,questionnaire_time_point,zipcode,number_adults,number_minors,age_oldest,age_youngest,family_status,parent1,parent2,...,CCQ_sore_throat,CCQ_cough,CCQ_chest_pain,abdominal_pain,diarrhea,vomiting,Number_sickdays_last_winter,child_age_months,sleep_duration_weekdays,sleep_duration_weekends
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,...,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
CGW4K85NWW,Z,NA,NA,NA,NA,NA,NA,NA,NA,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
CGW4K85NWW,Z,NA,NA,NA,NA,NA,NA,NA,NA,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
CGW4K85NWW,Z,NA,NA,NA,NA,NA,NA,NA,NA,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
CGW4K85NWW,Z,NA,NA,NA,NA,NA,NA,NA,NA,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
ECMQH4UTAE,Z,NA,NA,NA,NA,NA,NA,NA,NA,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
ECMQH4UTAE,Z,NA,NA,NA,NA,NA,NA,NA,NA,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


In [6]:
# Filter out rows where questionnaire_time_point is 'Z'
sosci <- sosci %>%
  dplyr::filter(questionnaire_time_point != "Z")

In [7]:
# Filter out results for child_number = 1 because this is the child of interest and the rest are just sibilings
sosci <- sosci %>%
    dplyr::filter(child_number == 1)

In [8]:
# Check the unique values in the questionnaire_time_point column
unique(sosci$questionnaire_time_point)

[1] "V0-2"  "Vx1-2" "Vx2-2" "Vx3-2" "Vx4-2" "Vx6-2" "Vx5-2" "V0"    "Vx"   
[10] "Vx2"   "Vx3"   "Vx4"   "Vx5"   "Vx6"   "Vx1"

In [9]:
# Count the number of rows where questionnaire_time_point equals 'Vx'
count_Vx <- sosci %>%
  filter(questionnaire_time_point == "Vx") %>%
  dplyr::summarise(count = n())

count_Vx

count
<int>
53


checked manually the time line for the serial_IDs having Vx and it is Vx1 missing for them so we wil change that

In [10]:
# Replace 'Vx' with 'Vx1' in the questionnaire_time_point column
sosci <- sosci %>%
  mutate(questionnaire_time_point = str_replace(questionnaire_time_point, "^Vx$", "Vx1"))


In [11]:
# Get the dimensions of the DataFrame
num_rows <- nrow(sosci)
num_cols <- ncol(sosci)

# Print the dimensions in a sentence
print(sprintf("The DataFrame has %d rows and %d columns.", num_rows, num_cols))

[1] "The DataFrame has 3100 rows and 50 columns."


In [12]:
# Count the number of rows per serial_ID
count_per_serial_ID <- sosci %>%
  dplyr::group_by(serial_ID) %>%
  dplyr::summarise(total_rows = n(),
            unique_questionnaire_time_points = n_distinct(questionnaire_time_point))


In [13]:
count_per_serial_ID$unique_questionnaire_time_points

[1]  7 14 14 14  7 14  9  9 14  7 14 14 14  7 14 14  7 14 14  7 14 14  7 13 14
 [26]  1 14  8 14  7  9 14  7  7  7  7 14  1 14  7 14  7 14  7 14  7  7  7  7  8
 [51]  7 14  7  7 14 14  7 14  7  7  7  7  7 14 14  7 14 14 14 13 14  7  8 14  2
 [76]  7 14  7 11 14 14  7 14 14  7 14 14  1 14 14 14  7  6 14 14  7  7  9 14 14
[101] 14 14 14 14  7  7 14 14 14  8  7 14  3  7  7 14  7  7  7  8 14 14  7 14 14
[126]  7  3 14 14  7  1  7  7  7 14  8  7  7  7  7  7  7 14 14  2  7  7 14  7  7
[151] 14  7 14 13 14  1 14  4 14  7 14  7  7  7  1 14  7 14  7  4  7  1  7  9  7
[176]  1 14 14 14 14  7  1 14  7 14 14  9  7  7  7  7  7 14 14 14  7 14 14  6 14
[201]  7  7 14 14 14  7 14 14  9 14 14  7  8 14 14 14 14  7 14 14 14  7 14  7 14
[226]  7 14 14 14  7 14  7  7 14 14 14  7  5 14 14 14  7  3 14  7 14 14  1  7  7
[251] 14 14  7  7  8 14 14  8  7 14  7  7  7 14  7  7  7 10 14  7  7  7  7  7  7
[276] 14 14  7 14 14  2  7 14  1 14  7  7  7  1 13 14 14  1  7 10  7  7  7 14 14
[301] 14 14  1

In [14]:
# Map the time points to their respective stages
secondary_outcomes_data <- sosci %>%
  mutate(time_point_label = case_when(
    questionnaire_time_point == 'V0' ~ 'Baseline',
    questionnaire_time_point == 'V0-2' ~ 'Follow',
    questionnaire_time_point == 'Vx1' ~ 'Baseline_week1',
    questionnaire_time_point == 'Vx1-2' ~ 'Follow_week1',
    questionnaire_time_point == 'Vx2' ~ 'Baseline_week2',
    questionnaire_time_point == 'Vx2-2' ~ 'Follow_week2',
    questionnaire_time_point == 'Vx3' ~ 'Baseline_week3',
    questionnaire_time_point == 'Vx3-2' ~ 'Follow_week3',
    questionnaire_time_point == 'Vx4' ~ 'Baseline_week4',
    questionnaire_time_point == 'Vx4-2' ~ 'Follow_week4',
    questionnaire_time_point == 'Vx5' ~ 'Baseline_week5',
    questionnaire_time_point == 'Vx5-2' ~ 'Follow_week5',
    questionnaire_time_point == 'Vx6' ~ 'Baseline_week6',
    questionnaire_time_point == 'Vx6-2' ~ 'Follow_week6',
    TRUE ~ questionnaire_time_point  # Leave other points as they are
  ))

In [15]:
unique(secondary_outcomes_data$kindergarten_ID)

[1] "admin" "MZA1"  "MZA2"  NA      "Admin" "MTB1"  "MTB2"  "MTB3"  "PBB1" 
[10] "PBB2"  "RGB1"  "RGB2"

In [16]:
# Link each kindergarten_ID to its respective group
secondary_outcomes_data <- secondary_outcomes_data %>%
  mutate(group = case_when(
    kindergarten_ID == 'MTB1' ~ 'Kneipp',
    kindergarten_ID == 'MTB2' ~ 'Control',
    kindergarten_ID == 'MTB3' ~ 'Control',
    kindergarten_ID == 'PBB1' ~ 'Kneipp',
    kindergarten_ID == 'PBB2' ~ 'Control',
    kindergarten_ID == 'RGB1' ~ 'Kneipp',
    kindergarten_ID == 'RGB2' ~ 'Control',
    kindergarten_ID == 'MZA1' ~ 'Kneipp',
    kindergarten_ID == 'MZA2' ~ 'Control',
    TRUE ~ 'Unknown'  # In case there's any unexpected ID
  ))

In [17]:
# Replace `kindergarten_ID == 'Admin'` with data based on `serial_ID` and `questionnaire_time_point`
secondary_outcomes_data <- secondary_outcomes_data %>%
  group_by(serial_ID, questionnaire_time_point) %>%
  mutate(kindergarten_ID = if_else(kindergarten_ID == 'Admin' & !is.na(lag(kindergarten_ID)),
                                   lag(kindergarten_ID), kindergarten_ID)) %>%
  ungroup()

In [18]:
# Replace `kindergarten_ID == 'Admin'` with data based on `serial_ID` and `questionnaire_time_point`
secondary_outcomes_data <- secondary_outcomes_data %>%
  group_by(serial_ID, questionnaire_time_point) %>%
  mutate(kindergarten_ID = if_else(kindergarten_ID == 'admin' & !is.na(lag(kindergarten_ID)),
                                   lag(kindergarten_ID), kindergarten_ID)) %>%
  ungroup()

In [19]:
# Create the 'pairs' variable
secondary_outcomes_data <- secondary_outcomes_data %>%
  mutate(pairs = case_when(
    kindergarten_ID %in% c('MTB1', 'MTB2', 'MTB3') ~ 'MTB_pair',
    kindergarten_ID %in% c('PBB1', 'PBB2') ~ 'PBB_pair',
    kindergarten_ID %in% c('RGB1', 'RGB2') ~ 'RGB_pair',
    kindergarten_ID %in% c('MZA1', 'MZA2') ~ 'MZA_pair',
    TRUE ~ NA_character_  # Keep other entries as NA
  ))

In [20]:
colnames(secondary_outcomes_data)

[1] "serial_ID"                     "questionnaire_time_point"     
 [3] "zipcode"                       "number_adults"                
 [5] "number_minors"                 "age_oldest"                   
 [7] "age_youngest"                  "family_status"                
 [9] "parent1"                       "parent2"                      
[11] "education_parent1"             "education_parent2"            
[13] "occupation_parent1"            "occupation_parent2"           
[15] "household_income"              "kindergarten_ID"              
[17] "child_number"                  "acclimatization"              
[19] "birth_order"                   "weight"                       
[21] "height"                        "sex"                          
[23] "vaccination_status"            "pre_existing_disease"         
[25] "chronic_disease"               "time_outside"                 
[27] "sport_and_club"                "cold_last_winter"             
[29] "bronchitis_last_winter"        "lung_inflammation_last_winter"
[31] "antibiotics_last_winter"       "GI_last_winter"               
[33] "sick_days_due_sickness"        "sick_days_due_to_other"       
[35] "CCQ_fever"                     "CCQ_chills"                   
[37] "CCQ_muscle_pain"               "CCQ_watery_eyes"              
[39] "CCQ_runny_nose"                "CCQ_sneezing"                 
[41] "CCQ_sore_throat"               "CCQ_cough"                    
[43] "CCQ_chest_pain"                "abdominal_pain"               
[45] "diarrhea"                      "vomiting"                     
[47] "Number_sickdays_last_winter"   "child_age_months"             
[49] "sleep_duration_weekdays"       "sleep_duration_weekends"      
[51] "time_point_label"              "group"                        
[53] "pairs"

BASELINE

In [21]:
# Define the symptoms and pre-intervention time points
ccq_symptoms <- c('CCQ_fever', 'CCQ_chills', 'CCQ_muscle_pain', 'CCQ_watery_eyes',
                  'CCQ_runny_nose', 'CCQ_sneezing', 'CCQ_sore_throat', 'CCQ_cough', 
                  'CCQ_chest_pain')
pre_intervention_time_points <- c('V0', 'Vx1', 'Vx2', 'Vx3', 'Vx4', 'Vx5', 'Vx6')


In [22]:
# Step 1: Create a sum score of the CCQ symptoms for each time point
secondary_outcomes_data <- secondary_outcomes_data %>%
  rowwise() %>%
  dplyr::mutate(CCQ_sum_score = sum(c_across(all_of(ccq_symptoms)), na.rm = TRUE)) %>%
  dplyr::ungroup()


In [23]:
# Step 2: Calculate the average CCQ score over pre-intervention time points
secondary_outcomes_data <- secondary_outcomes_data %>%
  dplyr::group_by(serial_ID) %>%
  dplyr::mutate(CCQ_baseline_score = mean(CCQ_sum_score[questionnaire_time_point %in% pre_intervention_time_points], na.rm = TRUE)) %>%
  dplyr::ungroup()

In [24]:
# Define the other baseline conditions
baseline_conditions <- c('cold_last_winter', 'bronchitis_last_winter', 
                         'lung_inflammation_last_winter', 'antibiotics_last_winter', 
                         'GI_last_winter')

In [25]:
# Step 3: Calculate the sum score and average for each baseline condition separately
secondary_outcomes_data <- secondary_outcomes_data %>%
  dplyr::group_by(serial_ID) %>%
  dplyr::mutate(baseline_cold_last_winter = mean(cold_last_winter[questionnaire_time_point %in% pre_intervention_time_points], na.rm = TRUE),
         baseline_bronchitis_last_winter = mean(bronchitis_last_winter[questionnaire_time_point %in% pre_intervention_time_points], na.rm = TRUE),
         baseline_lung_inflammation_last_winter = mean(lung_inflammation_last_winter[questionnaire_time_point %in% pre_intervention_time_points], na.rm = TRUE),
         baseline_antibiotics_last_winter = mean(antibiotics_last_winter[questionnaire_time_point %in% pre_intervention_time_points], na.rm = TRUE),
         baseline_GI_last_winter = mean(GI_last_winter[questionnaire_time_point %in% pre_intervention_time_points], na.rm = TRUE)) %>%
  dplyr::ungroup()


In [26]:
secondary_outcomes_data$CCQ_sum_score

[1]  0  0  0  9  0  9  0  9  9 14  9  9  0  0  0  0  0  0  0  0  0  0  0  0
  [25]  0  0  0  0  0  0  0  0  0  0  9 12 11 10 13 11  9  9 12  9 10 11  9  9
  [49]  9  9  9  0  0  0  9  9 12 12 12 13  9  9  9  9 10 13  9  9 17 10  9 19
  [73]  9  9 16 10  9  0 17  0  9  9  9 10  9 12  9  9 10  9 14  9 10  9 10 26
  [97] 10  9  9  9  0 10 10  9 12  9 12  9  0  9  9 11  0 10 16 12  9 11 15 13
 [121] 20 10 10  9 12 14 15  0  9  9 13  9 11 13  9 15 12 15  9 12  9 10  9 11
 [145] 12  9 13  9 12 13 13  9 13 12 12  9  9  0  9 12  9 11 10  9  9  9 12 22
 [169] 10 22  9  9  9  0  9  9  9  9  9  0  0  0  0  0  0  0  0  0  0  0  0  0
 [193]  0  0  0 10  9 15  9  9  9 15 15  9 23 11 36 10 11  9  9 12  0 19  9  9
 [217]  9  9 11 12 12  9 10  9  9  9 12 25 11 10  9 15  0  9 18 16  9  9 11  9
 [241] 11 10  9  9  9  9 19 17 13  9 10 12 12  9 12 10  9  9  9  9 11  9 21  0
 [265]  9  9  9  9  9 13 14  9  9  9 12 12  9 14 12  9  9 12  9 13 10 16  9  9
 [289] 10 17 10 13 11  9 11 19  9  9 17 12  9 11  9  9  9  9 11 10 13  9  9  9
 [313]  0  0  0  0  0  0  0  0  0  0  0  0 10  9 10 12 12  9 11  0  0 20  0  9
 [337] 16 11 10  9 18 36  9  0  0  0  9 13  0  0  0 11  0 14 10 11  0  0  9 11
 [361] 15  0  0  0  9  0 11  9 10 14 14  0  0  0  9  0  0 13 11  9 11 12 12  9
 [385] 17 11 11 13 11  9 12 11 12 15  9 17 12 17 13  9 11 11 12 11 13  9  0 13
 [409] 11  0  9  0  9  0  0  0  9 13 17  0  9 20  9 10  9 11  9 13 16 17  9 10
 [433] 14 10  9  9 10 13 10 10  9 21 12 12 14 13 10  9 21 13  9  9  0 13 10 25
 [457] 16 26 11 13 13 12  9  9  9 10  9 10  9 10 12 11 20  9 19 12  9 13 14 10
 [481] 11 11 11  9 11 11 12  9  9 11  9 12 11  0 10 18 11  9 12 18 10 18 14 12
 [505] 12  9 10 10 14  9 12  9 12  9  9 13 12  9 13 12  9 11 13 14 10 15 11  9
 [529] 10  0  0 11 13 12 12  9 11  9 18 13 11 14 16 12 18  9  9  9 10  9  9 16
 [553] 11  9 11 13  9 18 12  9 11  9  9 17 13 14  9  9  9  9  9  9  9  9 10 10
 [577]  9  9 18 10  9  0  9 15  0  9  9  9 14 10 11  9  9  9  9  9  9  9 17 16
 [601]  9  0  0  9  9  9 11 10  0  9  9  9  9 15 11  9 14  0  9 11  0  9  9  0
 [625]  9  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 10 10
 [649]  9 10 10 12  8 11 14 10 12  0  0  9  9  9 17  9  0 14  9  0 11 18 11  9
 [673]  9  9  9 10  9 14 10 14 14 14  9 13 12  9  9 12 10 13  9  9  9  9 12 10
 [697] 10  9  9 12 13  9 16  9 15  9  9  9 10 12 10 10  9 12  9 15 10  9 11  9
 [721]  9  9 10  9 12 10  9  9 10 10  9 10  9  9  9 13  9 10  9  9 13 11  9  9
 [745]  9  9  9  9 11 10 11 10  9 11  9  9  9  9 11 11  9  0 14  0  9  9  9  9
 [769]  9 11  9  9  0  0  0  0  0  0  0  0  0  0  0 10  9  9 10  9  9  9 10  9
 [793]  9  0  9 10  0  9  9  9  9  9  9 10 11  9  0  9  9 11  9 12  9  9  0  9
 [817]  9  9  9  9 16 10  9  9  9  9 11 12  9 13  9  9  9  9  9 12  9  9  9  9
 [841]  9  9  9 10  9  0  9  9  9 10  9  9  9  9  9  9  9 23  9  9 10 11 10  9
 [865]  9  9  9  9  9  9  9  9  9  9 12 15  9  9  9 12  0  0  9  9  0  0  0  0
 [889]  0  0  0  0  0  0  0  0  0  0  0  9 11 11  0 15 11 10  0  9 12  9 15 13
 [913] 11 19 13 10 19  9 11 15  9  9 10 10 13 13 11 11  9  9 12  9 10  9 12  0
 [937] 11  9 16  9  9 11  9 15 12  9  9  9 13 10 18 15 10  9 20  9 11 11  0  9
 [961]  9  9  9  9 15 15 11  9  9 11  9 10  9  9  9 11  9 13 14  0  9  9  9  9
 [985]  9 10 10  9 10 10  0  0  0  0  0  0  0  0  0  9 10 20 14 15  9 18  0 12
[1009]  9 14  0 16 16  9  9 12  9  9  0 10 12  9  9 18  9 12 15  9  9  9  9  9
[1033]  9  9 10  9 10 15 13 15  9 11  9  9  9 12 11  9 10  9  9  9 12  9 15 18
[1057]  9 13 18 14 12 15 15 14  9  9 17  9  0  0  0  0  0  0  0  0  0  0  0  0
[1081]  0  0  0  0  9  9 10  9  0 10  9  0  9 13  9 14 12 13  9 11  9  9 11  9
[1105] 12  9 11  9 15  9  9  9  9  9 11 10  9 11 10  9  9  9  9  9 12  0  9  9
[1129]  9 16 19  9 10 10  9  9  9  9  0  9 19 19 10 11 18  9  9 15 14 10  9 12
[1153] 15 13  9 15 11  9  9 16  9  9  9  9 10 11 10  9  9 12  9  9  9  9  9  9
[1177]  0  0  0  9  9  9  9 13  9  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
[1201]  0  0  0  0  0  0  0  0  0  0  0  0  0  0  9  

Follow up

In [27]:
# Define the post-intervention time points
post_intervention_time_points <- c('V0-2', 'Vx1-2', 'Vx2-2', 'Vx3-2', 'Vx4-2', 'Vx5-2', 'Vx6-2')

# Step 3: Calculate the average CCQ score over post-intervention time points
secondary_outcomes_data <- secondary_outcomes_data %>%
  dplyr::group_by(serial_ID) %>%
  dplyr::mutate(CCQ_followup_score = mean(CCQ_sum_score[questionnaire_time_point %in% post_intervention_time_points], na.rm = TRUE)) %>%
  dplyr::ungroup()

In [28]:
# Step 5: Calculate the sum score and average for each baseline condition separately over post-intervention time points
secondary_outcomes_data <- secondary_outcomes_data %>%
  dplyr::group_by(serial_ID) %>%
  dplyr::mutate(followup_cold_last_winter = mean(cold_last_winter[questionnaire_time_point %in% post_intervention_time_points], na.rm = TRUE),
         followup_bronchitis_last_winter = mean(bronchitis_last_winter[questionnaire_time_point %in% post_intervention_time_points], na.rm = TRUE),
         followup_lung_inflammation_last_winter = mean(lung_inflammation_last_winter[questionnaire_time_point %in% post_intervention_time_points], na.rm = TRUE),
         followup_antibiotics_last_winter = mean(antibiotics_last_winter[questionnaire_time_point %in% post_intervention_time_points], na.rm = TRUE),
         followup_GI_last_winter = mean(GI_last_winter[questionnaire_time_point %in% post_intervention_time_points], na.rm = TRUE)) %>%
  dplyr::ungroup()

In [29]:
secondary_outcomes_data$CCQ_followup_score

[1]  7.000000  0.000000  7.714286  7.714286  0.000000  7.714286  0.000000
   [8]  7.714286  7.714286  7.000000  7.714286  7.714286  9.714286  9.285714
  [15]  9.000000  8.857143  7.714286 10.142857 10.714286  3.000000  8.571429
  [22]  9.714286  9.428571  9.571429 10.571429 13.857143  7.857143 12.714286
  [29]  7.714286  8.000000  9.571429  8.142857  7.857143  8.428571  7.714286
  [36]  9.714286  9.428571  9.285714 13.857143  9.571429  8.571429  9.000000
  [43] 10.142857 10.714286 10.571429  9.571429  7.714286  8.857143  7.857143
  [50]  3.000000  8.000000  0.000000  3.400000  3.400000  7.857143  8.428571
  [57]  9.714286 12.714286 12.714286 10.142857  9.000000  8.428571  8.857143
  [64]  7.714286  9.285714 10.714286  7.714286  9.428571 10.571429  9.571429
  [71]  7.857143  9.714286  8.571429  9.571429 13.857143  8.142857  7.857143
  [78]  0.000000  3.400000  3.000000  8.000000  8.142857  9.714286  8.142857
  [85]  7.857143 10.142857  7.714286  9.714286  9.428571  8.428571  9.000000
  [92]  9.285714  9.571429  7.714286 10.571429 13.857143  8.000000  8.857143
  [99]  8.571429  9.571429  0.000000  9.714286  9.571429  7.857143  9.428571
 [106]  7.714286  9.285714  7.714286  0.000000  7.857143  7.857143  8.428571
 [113]  3.400000 10.714286 10.714286 12.714286 12.714286 10.142857 10.571429
 [120]  8.857143  9.714286  8.000000  8.142857  9.000000  8.571429  9.571429
 [127] 13.857143  0.000000  9.714286  7.714286  9.571429  8.142857  9.285714
 [134] 10.571429  9.714286 10.714286  9.428571 13.857143  7.714286 10.142857
 [141]  9.000000  8.857143  8.000000  9.571429  8.571429  7.857143  9.000000
 [148]  7.714286  8.857143  9.571429  9.285714  9.714286  9.571429  9.428571
 [155] 13.857143  7.714286  8.571429  3.400000  8.142857 10.714286 10.571429
 [162] 10.142857  7.857143  8.000000  7.857143  8.428571  8.428571 12.714286
 [169]  7.857143 12.714286  9.714286  9.714286  7.714286  7.714286  7.714286
 [176]  7.714286  7.714286  7.714286  7.714286 15.571429  8.000000 10.857143
 [183] 10.714286  8.571429 10.857143  7.857143  7.714286 10.000000 13.285714
 [190]  8.857143  8.857143  9.142857 11.285714  8.142857  8.142857  7.857143
 [197]  8.000000 10.857143  9.142857  8.142857  7.714286 10.857143 10.714286
 [204]  8.857143 11.285714 13.285714 15.571429  8.857143  8.571429  8.142857
 [211] 10.000000  8.714286  9.571429  9.571429 10.000000  8.000000  8.142857
 [218]  8.857143  9.571429 10.857143 10.714286  8.857143 10.857143 15.571429
 [225]  7.714286  7.857143  9.142857 13.285714  8.571429  8.857143  8.142857
 [232] 11.285714  8.714286  8.714286 13.285714 10.857143  9.142857  8.142857
 [239]  8.000000  7.857143 10.714286 10.857143  8.142857  7.714286  9.571429
 [246]  8.857143 10.000000 15.571429  8.714286  8.857143  8.571429 10.000000
 [253]  8.142857  9.142857 10.857143 10.857143  8.857143  7.714286  8.714286
 [260]  8.000000 11.285714 11.285714 15.571429  8.857143  7.857143  8.857143
 [267]  8.857143  8.857143  8.571429 10.714286 13.285714  8.142857  9.571429
 [274]  8.857143 10.000000 10.857143  8.714286 13.285714  9.142857  8.142857
 [281]  8.000000  8.857143  7.714286 10.714286 10.857143  8.857143  7.857143
 [288]  9.571429  8.142857 15.571429  8.571429  9.142857  8.142857  8.000000
 [295] 10.857143 10.857143  8.714286  8.142857  8.857143 11.285714 11.285714
 [302] 13.285714  8.571429 15.571429  7.857143  7.714286 10.714286  9.571429
 [309]  8.857143  8.857143  8.857143 10.000000       NaN  9.714286 10.142857
 [316]       NaN 10.857143 10.857143  0.000000       NaN       NaN 13.857143
 [323]  7.857143 10.000000       NaN       NaN  7.857143 10.142857       NaN
 [330]       NaN       NaN  9.142857  7.714286  9.142857 10.571429 13.857143
 [337] 10.571429 10.857143       NaN  7.714286  7.714286  7.714286  9.571429
 [344]  8.571429  9.571429 15.571429  9.714286 10.000000       NaN  9.571429
 [351]  8.571429  8.571429       NaN       NaN  0.000000  9.571429 13.285714
 [358]  9.571429  9.571429 13.285714 10.857143       NaN  7.714286  7.714286

In [30]:
# Define the time points to be removed
time_points_to_remove <- c('Vx1-2', 'Vx2-2', 'Vx3-2', 'Vx4-2', 'Vx5-2', 'Vx6-2', 
                           'Vx1', 'Vx2', 'Vx3', 'Vx4', 'Vx5', 'Vx6')

# Remove the rows with the defined time points
secondary_outcomes_data <- secondary_outcomes_data %>%
  filter(!questionnaire_time_point %in% time_points_to_remove)

In [31]:
unique(secondary_outcomes_data$questionnaire_time_point)

[1] "V0-2" "V0"

In [32]:
# Separate Intention-to-Treat (ITT) population - includes all participants
ITT_population <- secondary_outcomes_data

# Check the number of participants in each population
cat("Number of participants in ITT population:", n_distinct(ITT_population$serial_ID), "\n")


Number of participants in ITT population: 300 


In [33]:
# Count the number of males and females by the 'group' variable
sex_distribution <- ITT_population %>%
  group_by(group, sex) %>%
  dplyr::summarise(count = n(), .groups = 'drop')

# Display the result
print(sex_distribution)

# A tibble: 10 x 3
   group     sex count
   <chr>   <dbl> <int>
 1 Control     1    62
 2 Control     2    49
 3 Control    NA     4
 4 Kneipp      1    52
 5 Kneipp      2    39
 6 Kneipp     NA    14
 7 Unknown     1   128
 8 Unknown     2   124
 9 Unknown     3     1
10 Unknown    NA     3


In [34]:
control = 56 + 40 + 3
kneipp = 52 + 39 + 14

control
kneipp

[1] 99

[1] 105

CCQ

In [35]:
library(lme4)

Loading required package: Matrix


Attaching package: 'Matrix'


The following objects are masked from 'package:tidyr':

    expand, pack, unpack




In [36]:
# Model for Male participants
male_model <- lmer(CCQ_followup_score ~ group + CCQ_baseline_score + (1 | pairs) + (1 | kindergarten_ID),
                   data = ITT_population %>% filter(sex == "1"))

# Model for Female participants
female_model <- lmer(CCQ_followup_score ~ group + CCQ_baseline_score + (1 | pairs) + (1 | kindergarten_ID),
                     data = ITT_population %>% filter(sex == "2"))


boundary (singular) fit: see help('isSingular')



In [37]:
# Step 2: Summary of the models
cat("Results for Male participants:\n")
summary(male_model)

# Confidence Intervals for Male Model
confint(male_model, level = 0.95)

Results for Male participants:


Linear mixed model fit by REML ['lmerMod']
Formula: CCQ_followup_score ~ group + CCQ_baseline_score + (1 | pairs) +  
    (1 | kindergarten_ID)
   Data: ITT_population %>% filter(sex == "1")

REML criterion at convergence: 355.3

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-4.3635 -0.4476 -0.0340  0.5156  3.3343 

Random effects:
 Groups          Name        Variance Std.Dev.
 kindergarten_ID (Intercept) 0.000    0.000   
 pairs           (Intercept) 0.000    0.000   
 Residual                    4.462    2.112   
Number of obs: 82, groups:  kindergarten_ID, 9; pairs, 4

Fixed effects:
                   Estimate Std. Error t value
(Intercept)          2.4675     1.2817   1.925
groupKneipp         -0.7986     0.4687  -1.704
CCQ_baseline_score   0.7714     0.1359   5.676

Correlation of Fixed Effects:
            (Intr) grpKnp
groupKneipp -0.108       
CCQ_bsln_sc -0.969 -0.063
optimizer (nloptwrap) convergence code: 0 (OK)
boundary (singular) fit: see help('isSingular')

Computing profile confidence intervals ...



,2.5 %,97.5 %
.sig01,0.00000000,0.6422715
.sig02,0.00000000,0.6646955
.sigma,1.79235054,2.4361506
(Intercept),-0.02736999,4.9624083
groupKneipp,-1.71091274,0.1205583
CCQ_baseline_score,0.50684638,1.0359010


In [38]:
cat("\n\nResults for Female participants:\n")
summary(female_model)

# Confidence Intervals for Female Model
confint(female_model, level = 0.95)



Results for Female participants:


Linear mixed model fit by REML ['lmerMod']
Formula: CCQ_followup_score ~ group + CCQ_baseline_score + (1 | pairs) +  
    (1 | kindergarten_ID)
   Data: ITT_population %>% filter(sex == "2")

REML criterion at convergence: 247.4

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-3.4781 -0.4183 -0.0933  0.7272  1.5962 

Random effects:
 Groups          Name        Variance Std.Dev.
 kindergarten_ID (Intercept) 0.05208  0.2282  
 pairs           (Intercept) 0.49732  0.7052  
 Residual                    2.75430  1.6596  
Number of obs: 63, groups:  kindergarten_ID, 9; pairs, 4

Fixed effects:
                   Estimate Std. Error t value
(Intercept)         7.22136    1.12397   6.425
groupKneipp        -0.04147    0.45892  -0.090
CCQ_baseline_score  0.15004    0.10842   1.384

Correlation of Fixed Effects:
            (Intr) grpKnp
groupKneipp -0.220       
CCQ_bsln_sc -0.901  0.036

Computing profile confidence intervals ...

Warning message in nextpar(mat, cc, i, delta, lowcut, upcut):
"unexpected decrease in profile: using minstep"
Warning message in profile.merMod(object, which = parm, signames = oldNames, ...):
"non-monotonic profile for (Intercept)"
Warning message in confint.thpr(pp, level = level, zeta = zeta):
"bad spline fit for (Intercept): falling back to linear interpolation"


,2.5 %,97.5 %
.sig01,0.00000000,1.2123416
.sig02,0.00000000,1.8330077
.sigma,1.38092102,1.9848609
(Intercept),5.10134870,9.4674301
groupKneipp,-0.94581906,1.2284480
CCQ_baseline_score,-0.06726918,0.3585418


In [39]:
# For Kneipp group
kneipp_stats <- ITT_population %>%
  dplyr::group_by(group, sex) %>%
  dplyr::summarise(
    n = n(),
    mean_CCQ = mean(CCQ_followup_score, na.rm = TRUE),
    sd_CCQ = sd(CCQ_followup_score, na.rm = TRUE),
    min_CCQ = min(CCQ_followup_score, na.rm = TRUE),
    max_CCQ = max(CCQ_followup_score, na.rm = TRUE)
  )

# Display the result
print(kneipp_stats)

Warning message:
"There were 2 warnings in `dplyr::summarise()`.
The first warning was:
i In argument: `min_CCQ = min(CCQ_followup_score, na.rm = TRUE)`.
i In group 9: `group = "Unknown"`, `sex = 3`.
Caused by warning in `min()`:
! no non-missing arguments to min; returning Inf
i Run `dplyr::last_dplyr_warnings()` to see the 1 remaining warning."
`summarise()` has grouped output by 'group'. You can override using the
`.groups` argument.


# A tibble: 10 x 7
# Groups:   group [3]
   group     sex     n mean_CCQ sd_CCQ min_CCQ max_CCQ
   <chr>   <dbl> <int>    <dbl>  <dbl>   <dbl>   <dbl>
 1 Control     1    62     9.71   2.46    0       16.7
 2 Control     2    49     8.95   1.85    2.25    13.1
 3 Control    NA     4     0      0       0        0  
 4 Kneipp      1    52     9.11   2.47    0       13.9
 5 Kneipp      2    39     8.93   1.67    3.4     12.7
 6 Kneipp     NA    14     0      0       0        0  
 7 Unknown     1   128     8.50   3.44    0       16.7
 8 Unknown     2   124     7.91   3.16    0       11.7
 9 Unknown     3     1   NaN     NA     Inf     -Inf  
10 Unknown    NA     3     4.5   NA       4.5      4.5


COLD

In [40]:
# Model for followup_cold_last_winter by sex
cold_male_model <- lmer(followup_cold_last_winter ~ group + baseline_cold_last_winter + (1 | pairs) + (1 | kindergarten_ID),
                        data = ITT_population %>% filter(sex == "1"))

cold_female_model <- lmer(followup_cold_last_winter ~ group + baseline_cold_last_winter + (1 | pairs) + (1 | kindergarten_ID),
                          data = ITT_population %>% filter(sex == "2"))


boundary (singular) fit: see help('isSingular')



In [41]:
summary(cold_male_model)
confint(cold_male_model)


Linear mixed model fit by REML ['lmerMod']
Formula: followup_cold_last_winter ~ group + baseline_cold_last_winter +  
    (1 | pairs) + (1 | kindergarten_ID)
   Data: ITT_population %>% filter(sex == "1")

REML criterion at convergence: 70

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-2.6175  0.1092  0.3109  0.4188  1.2439 

Random effects:
 Groups          Name        Variance Std.Dev.
 kindergarten_ID (Intercept) 0.019679 0.14028 
 pairs           (Intercept) 0.008844 0.09404 
 Residual                    0.116614 0.34149 
Number of obs: 81, groups:  kindergarten_ID, 9; pairs, 4

Fixed effects:
                          Estimate Std. Error t value
(Intercept)                 1.3352     0.2746   4.862
groupKneipp                 0.0909     0.1266   0.718
baseline_cold_last_winter   0.2123     0.1336   1.589

Correlation of Fixed Effects:
            (Intr) grpKnp
groupKneipp -0.250       
bsln_cld_l_ -0.932  0.034

Computing profile confidence intervals ...



,2.5 %,97.5 %
.sig01,0.00000000,0.2918549
.sig02,0.00000000,0.3274480
.sigma,0.29101883,0.4051113
(Intercept),0.81707812,1.8809295
groupKneipp,-0.15759770,0.3733551
baseline_cold_last_winter,-0.04790029,0.4760725


In [42]:
summary(cold_female_model)
confint(cold_female_model)

Linear mixed model fit by REML ['lmerMod']
Formula: followup_cold_last_winter ~ group + baseline_cold_last_winter +  
    (1 | pairs) + (1 | kindergarten_ID)
   Data: ITT_population %>% filter(sex == "2")

REML criterion at convergence: 9.8

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-3.7815 -0.0598  0.3103  0.3103  2.2853 

Random effects:
 Groups          Name        Variance Std.Dev.
 kindergarten_ID (Intercept) 0.00000  0.0000  
 pairs           (Intercept) 0.00000  0.0000  
 Residual                    0.05973  0.2444  
Number of obs: 62, groups:  kindergarten_ID, 9; pairs, 4

Fixed effects:
                          Estimate Std. Error t value
(Intercept)                0.77790    0.20493   3.796
groupKneipp                0.09047    0.06444   1.404
baseline_cold_last_winter  0.57313    0.10175   5.633

Correlation of Fixed Effects:
            (Intr) grpKnp
groupKneipp -0.402       
bsln_cld_l_ -0.978  0.266
optimizer (nloptwrap) convergence code: 0 (OK)
boundar

Computing profile confidence intervals ...



,2.5 %,97.5 %
.sig01,0.00000000,0.08806251
.sig02,0.00000000,0.09941377
.sigma,0.20188588,0.28741704
(Intercept),0.37993473,1.17586159
groupKneipp,-0.03470887,0.21564374
baseline_cold_last_winter,0.37553388,0.77072830


In [43]:
# For Kneipp group
kneipp_stats <- ITT_population %>%
  dplyr::group_by(group, sex) %>%
  dplyr::summarise(
    n = n(),
    mean_CCQ = mean(followup_cold_last_winter, na.rm = TRUE),
    sd_CCQ = sd(followup_cold_last_winter, na.rm = TRUE),
    min_CCQ = min(followup_cold_last_winter, na.rm = TRUE),
    max_CCQ = max(followup_cold_last_winter, na.rm = TRUE)
  )

# Display the result
print(kneipp_stats)

Warning message:
"There were 8 warnings in `dplyr::summarise()`.
The first warning was:
i In argument: `min_CCQ = min(followup_cold_last_winter, na.rm = TRUE)`.
i In group 3: `group = "Control"`, `sex = NA`.
Caused by warning in `min()`:
! no non-missing arguments to min; returning Inf
i Run `dplyr::last_dplyr_warnings()` to see the 7 remaining warnings."
`summarise()` has grouped output by 'group'. You can override using the
`.groups` argument.


# A tibble: 10 x 7
# Groups:   group [3]
   group     sex     n mean_CCQ sd_CCQ min_CCQ max_CCQ
   <chr>   <dbl> <int>    <dbl>  <dbl>   <dbl>   <dbl>
 1 Control     1    62     1.87  0.338       1       2
 2 Control     2    49     1.92  0.277       1       2
 3 Control    NA     4   NaN    NA         Inf    -Inf
 4 Kneipp      1    52     1.85  0.364       1       2
 5 Kneipp      2    39     1.92  0.270       1       2
 6 Kneipp     NA    14   NaN    NA         Inf    -Inf
 7 Unknown     1   128     1.86  0.352       1       2
 8 Unknown     2   124     1.86  0.344       1       2
 9 Unknown     3     1   NaN    NA         Inf    -Inf
10 Unknown    NA     3   NaN    NA         Inf    -Inf


BRONCHITIS

In [44]:
# Model for followup_bronchitis_last_winter by sex
bronchitis_male_model <- lmer(followup_bronchitis_last_winter ~ group + baseline_bronchitis_last_winter + (1 | pairs) + (1 | kindergarten_ID),
                              data = ITT_population %>% filter(sex == "1"))

bronchitis_female_model <- lmer(followup_bronchitis_last_winter ~ group + baseline_bronchitis_last_winter + (1 | pairs) + (1 | kindergarten_ID),
                                data = ITT_population %>% filter(sex == "2"))


boundary (singular) fit: see help('isSingular')

boundary (singular) fit: see help('isSingular')



In [45]:
summary(bronchitis_male_model)
confint(bronchitis_male_model)


Linear mixed model fit by REML ['lmerMod']
Formula: 
followup_bronchitis_last_winter ~ group + baseline_bronchitis_last_winter +  
    (1 | pairs) + (1 | kindergarten_ID)
   Data: ITT_population %>% filter(sex == "1")

REML criterion at convergence: 66.5

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-1.5699 -0.4280 -0.1675 -0.0127  2.7962 

Random effects:
 Groups          Name        Variance Std.Dev.
 kindergarten_ID (Intercept) 0.000000 0.00000 
 pairs           (Intercept) 0.004994 0.07067 
 Residual                    0.118281 0.34392 
Number of obs: 81, groups:  kindergarten_ID, 9; pairs, 4

Fixed effects:
                                Estimate Std. Error t value
(Intercept)                      0.67974    0.12997   5.230
groupKneipp                      0.10887    0.07733   1.408
baseline_bronchitis_last_winter  0.37345    0.09454   3.950

Correlation of Fixed Effects:
            (Intr) grpKnp
groupKneipp -0.221       
bsln_brnc__ -0.865 -0.068
optimizer (nlopt

Computing profile confidence intervals ...

Warning message in nextpar(mat, cc, i, delta, lowcut, upcut):
"unexpected decrease in profile: using minstep"
Warning message in FUN(X[[i]], ...):
"non-monotonic profile for .sig01"
Warning message in nextpar(mat, cc, i, delta, lowcut, upcut):
"Last two rows have identical or NA .zeta values: using minstep"
Warning message in FUN(X[[i]], ...):
"non-monotonic profile for .sig02"
Warning message in confint.thpr(pp, level = level, zeta = zeta):
"bad spline fit for .sig01: falling back to linear interpolation"
Warning message in confint.thpr(pp, level = level, zeta = zeta):
"bad spline fit for .sig02: falling back to linear interpolation"
Warning message in regularize.values(x, y, ties, missing(ties), na.rm = na.rm):
"collapsing to unique 'x' values"


,2.5 %,97.5 %
.sig01,0.0000000,0.1544372
.sig02,0.0000000,0.2337881
.sigma,0.2949281,0.4016260
(Intercept),0.4382769,0.9215465
groupKneipp,-0.0444284,0.2585564
baseline_bronchitis_last_winter,0.1816857,0.5523098


In [46]:
summary(bronchitis_female_model)
confint(bronchitis_female_model, method = "Wald")

Linear mixed model fit by REML ['lmerMod']
Formula: 
followup_bronchitis_last_winter ~ group + baseline_bronchitis_last_winter +  
    (1 | pairs) + (1 | kindergarten_ID)
   Data: ITT_population %>% filter(sex == "2")

REML criterion at convergence: 2.9

Scaled residuals: 
     Min       1Q   Median       3Q      Max 
-2.02978 -0.01098  0.08065  0.09807  2.94628 

Random effects:
 Groups          Name        Variance  Std.Dev. 
 kindergarten_ID (Intercept) 5.282e-02 2.298e-01
 pairs           (Intercept) 8.559e-12 2.925e-06
 Residual                    4.176e-02 2.044e-01
Number of obs: 62, groups:  kindergarten_ID, 9; pairs, 4

Fixed effects:
                                Estimate Std. Error t value
(Intercept)                      0.60417    0.14292   4.228
groupKneipp                      0.15645    0.16855   0.928
baseline_bronchitis_last_winter  0.41793    0.07675   5.445

Correlation of Fixed Effects:
            (Intr) grpKnp
groupKneipp -0.530       
bsln_brnc__ -0.631  0.031

,2.5 %,97.5 %
.sig01,NA,NA
.sig02,NA,NA
.sigma,NA,NA
(Intercept),0.3240660,0.8842830
groupKneipp,-0.1739061,0.4868118
baseline_bronchitis_last_winter,0.2674890,0.5683626


In [47]:
# For Kneipp group
kneipp_stats <- ITT_population %>%
  dplyr::group_by(group, sex) %>%
  dplyr::summarise(
    n = n(),
    mean_CCQ = mean(followup_bronchitis_last_winter, na.rm = TRUE),
    sd_CCQ = sd(followup_bronchitis_last_winter, na.rm = TRUE),
    min_CCQ = min(followup_bronchitis_last_winter, na.rm = TRUE),
    max_CCQ = max(followup_bronchitis_last_winter, na.rm = TRUE)
  )

# Display the result
print(kneipp_stats)

Warning message:
"There were 8 warnings in `dplyr::summarise()`.
The first warning was:
i In argument: `min_CCQ = min(followup_bronchitis_last_winter, na.rm = TRUE)`.
i In group 3: `group = "Control"`, `sex = NA`.
Caused by warning in `min()`:
! no non-missing arguments to min; returning Inf
i Run `dplyr::last_dplyr_warnings()` to see the 7 remaining warnings."
`summarise()` has grouped output by 'group'. You can override using the
`.groups` argument.


# A tibble: 10 x 7
# Groups:   group [3]
   group     sex     n mean_CCQ sd_CCQ min_CCQ max_CCQ
   <chr>   <dbl> <int>    <dbl>  <dbl>   <dbl>   <dbl>
 1 Control     1    62     1.13  0.338       1       2
 2 Control     2    49     1.08  0.277       1       2
 3 Control    NA     4   NaN    NA         Inf    -Inf
 4 Kneipp      1    52     1.29  0.457       1       2
 5 Kneipp      2    39     1.10  0.307       1       2
 6 Kneipp     NA    14   NaN    NA         Inf    -Inf
 7 Unknown     1   128     1.16  0.373       1       2
 8 Unknown     2   124     1.12  0.329       1       2
 9 Unknown     3     1   NaN    NA         Inf    -Inf
10 Unknown    NA     3   NaN    NA         Inf    -Inf


LUNG INFLAMMATION

In [48]:
# Model for followup_lung_inflammation_last_winter by sex
lung_inflammation_male_model <- lmer(followup_lung_inflammation_last_winter ~ group + baseline_lung_inflammation_last_winter + (1 | pairs) + (1 | kindergarten_ID),
                                     data = ITT_population %>% filter(sex == "1"))

lung_inflammation_female_model <- lmer(followup_lung_inflammation_last_winter ~ group + baseline_lung_inflammation_last_winter + (1 | pairs) + (1 | kindergarten_ID),
                                       data = ITT_population %>% filter(sex == "2"))


boundary (singular) fit: see help('isSingular')

boundary (singular) fit: see help('isSingular')



In [49]:
summary(lung_inflammation_male_model)
confint(lung_inflammation_male_model, method = "Wald")

Linear mixed model fit by REML ['lmerMod']
Formula: 
followup_lung_inflammation_last_winter ~ group + baseline_lung_inflammation_last_winter +  
    (1 | pairs) + (1 | kindergarten_ID)
   Data: ITT_population %>% filter(sex == "1")

REML criterion at convergence: -82.5

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-3.4066 -0.1393 -0.1393  0.0647  6.9818 

Random effects:
 Groups          Name        Variance Std.Dev.
 kindergarten_ID (Intercept) 0.001057 0.03251 
 pairs           (Intercept) 0.000000 0.00000 
 Residual                    0.017738 0.13318 
Number of obs: 81, groups:  kindergarten_ID, 9; pairs, 4

Fixed effects:
                                       Estimate Std. Error t value
(Intercept)                             0.56114    0.10431   5.380
groupKneipp                            -0.05825    0.03838  -1.518
baseline_lung_inflammation_last_winter  0.48284    0.09764   4.945

Correlation of Fixed Effects:
            (Intr) grpKnp
groupKneipp -0.184       

,2.5 %,97.5 %
.sig01,NA,NA
.sig02,NA,NA
.sigma,NA,NA
(Intercept),0.3567028,0.76558184
groupKneipp,-0.1334700,0.01697404
baseline_lung_inflammation_last_winter,0.2914797,0.67420611


In [50]:
summary(lung_inflammation_female_model)
confint(lung_inflammation_female_model, method = "Wald")

Linear mixed model fit by REML ['lmerMod']
Formula: 
followup_lung_inflammation_last_winter ~ group + baseline_lung_inflammation_last_winter +  
    (1 | pairs) + (1 | kindergarten_ID)
   Data: ITT_population %>% filter(sex == "2")

REML criterion at convergence: -89.8

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-3.2211 -0.1006 -0.1006  0.1040  6.2376 

Random effects:
 Groups          Name        Variance Std.Dev.
 kindergarten_ID (Intercept) 0.00000  0.0000  
 pairs           (Intercept) 0.00000  0.0000  
 Residual                    0.01118  0.1057  
Number of obs: 62, groups:  kindergarten_ID, 9; pairs, 4

Fixed effects:
                                       Estimate Std. Error t value
(Intercept)                             0.68072    0.06921   9.836
groupKneipp                            -0.02163    0.02693  -0.803
baseline_lung_inflammation_last_winter  0.32991    0.06272   5.260

Correlation of Fixed Effects:
            (Intr) grpKnp
groupKneipp -0.253       

,2.5 %,97.5 %
.sig01,NA,NA
.sig02,NA,NA
.sigma,NA,NA
(Intercept),0.54507448,0.81636247
groupKneipp,-0.07440885,0.03115372
baseline_lung_inflammation_last_winter,0.20699022,0.45283383


In [51]:
# For Kneipp group
kneipp_stats <- ITT_population %>%
  dplyr::group_by(group, sex) %>%
  dplyr::summarise(
    n = n(),
    mean_CCQ = mean(followup_lung_inflammation_last_winter, na.rm = TRUE),
    sd_CCQ = sd(followup_lung_inflammation_last_winter, na.rm = TRUE),
    min_CCQ = min(followup_lung_inflammation_last_winter, na.rm = TRUE),
    max_CCQ = max(followup_lung_inflammation_last_winter, na.rm = TRUE)
  )

# Display the result
print(kneipp_stats)

Warning message:
"There were 8 warnings in `dplyr::summarise()`.
The first warning was:
i In argument: `min_CCQ = min(followup_lung_inflammation_last_winter, na.rm =
  TRUE)`.
i In group 3: `group = "Control"`, `sex = NA`.
Caused by warning in `min()`:
! no non-missing arguments to min; returning Inf
i Run `dplyr::last_dplyr_warnings()` to see the 7 remaining warnings."
`summarise()` has grouped output by 'group'. You can override using the
`.groups` argument.


# A tibble: 10 x 7
# Groups:   group [3]
   group     sex     n mean_CCQ sd_CCQ min_CCQ max_CCQ
   <chr>   <dbl> <int>    <dbl>  <dbl>   <dbl>   <dbl>
 1 Control     1    62     1.03  0.178       1       2
 2 Control     2    49     1.02  0.143       1       2
 3 Control    NA     4   NaN    NA         Inf    -Inf
 4 Kneipp      1    52     1.06  0.235       1       2
 5 Kneipp      2    39     1     0           1       1
 6 Kneipp     NA    14   NaN    NA         Inf    -Inf
 7 Unknown     1   128     1.02  0.147       1       2
 8 Unknown     2   124     1.03  0.163       1       2
 9 Unknown     3     1   NaN    NA         Inf    -Inf
10 Unknown    NA     3   NaN    NA         Inf    -Inf


Antibiotics

In [52]:
# Model for followup_antibiotics_last_winter by sex
antibiotics_male_model <- lmer(followup_antibiotics_last_winter ~ group + baseline_antibiotics_last_winter + (1 | pairs) + (1 | kindergarten_ID),
                               data = ITT_population %>% filter(sex == "1"))

antibiotics_female_model <- lmer(followup_antibiotics_last_winter ~ group + baseline_antibiotics_last_winter + (1 | pairs) + (1 | kindergarten_ID),
                                 data = ITT_population %>% filter(sex == "2"))


boundary (singular) fit: see help('isSingular')

boundary (singular) fit: see help('isSingular')



In [53]:
summary(antibiotics_male_model)
confint(antibiotics_male_model, method = "Wald")

Linear mixed model fit by REML ['lmerMod']
Formula: 
followup_antibiotics_last_winter ~ group + baseline_antibiotics_last_winter +  
    (1 | pairs) + (1 | kindergarten_ID)
   Data: ITT_population %>% filter(sex == "1")

REML criterion at convergence: 45.1

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-1.3786 -0.4304 -0.1379 -0.1379  3.1484 

Random effects:
 Groups          Name        Variance Std.Dev.
 kindergarten_ID (Intercept) 0.0000   0.0000  
 pairs           (Intercept) 0.0000   0.0000  
 Residual                    0.0926   0.3043  
Number of obs: 81, groups:  kindergarten_ID, 9; pairs, 4

Fixed effects:
                                 Estimate Std. Error t value
(Intercept)                       0.75341    0.13192   5.711
groupKneipp                       0.08902    0.06790   1.311
baseline_antibiotics_last_winter  0.28854    0.11338   2.545

Correlation of Fixed Effects:
            (Intr) grpKnp
groupKneipp -0.208       
bsln_ntbt__ -0.938 -0.029
optimizer 

,2.5 %,97.5 %
.sig01,NA,NA
.sig02,NA,NA
.sigma,NA,NA
(Intercept),0.49486046,1.0119608
groupKneipp,-0.04407228,0.2221077
baseline_antibiotics_last_winter,0.06632889,0.5107516


In [54]:
summary(antibiotics_female_model)
confint(antibiotics_female_model, method = "Wald")

Linear mixed model fit by REML ['lmerMod']
Formula: 
followup_antibiotics_last_winter ~ group + baseline_antibiotics_last_winter +  
    (1 | pairs) + (1 | kindergarten_ID)
   Data: ITT_population %>% filter(sex == "2")

REML criterion at convergence: -28.1

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-2.2838 -0.1646 -0.1646  0.1877  3.7168 

Random effects:
 Groups          Name        Variance Std.Dev.
 kindergarten_ID (Intercept) 0.00000  0.000   
 pairs           (Intercept) 0.00000  0.000   
 Residual                    0.03135  0.177   
Number of obs: 62, groups:  kindergarten_ID, 9; pairs, 4

Fixed effects:
                                 Estimate Std. Error t value
(Intercept)                       0.59157    0.08441   7.008
groupKneipp                       0.06238    0.04525   1.379
baseline_antibiotics_last_winter  0.37520    0.06689   5.609

Correlation of Fixed Effects:
            (Intr) grpKnp
groupKneipp -0.355       
bsln_ntbt__ -0.929  0.106
optimizer

,2.5 %,97.5 %
.sig01,NA,NA
.sig02,NA,NA
.sigma,NA,NA
(Intercept),0.42613019,0.7570061
groupKneipp,-0.02630285,0.1510705
baseline_antibiotics_last_winter,0.24408450,0.5063059


In [55]:
# For Kneipp group
kneipp_stats <- ITT_population %>%
  dplyr::group_by(group, sex) %>%
  dplyr::summarise(
    n = n(),
    mean_CCQ = mean(followup_antibiotics_last_winter, na.rm = TRUE),
    sd_CCQ = sd(followup_antibiotics_last_winter, na.rm = TRUE),
    min_CCQ = min(followup_antibiotics_last_winter, na.rm = TRUE),
    max_CCQ = max(followup_antibiotics_last_winter, na.rm = TRUE)
  )

# Display the result
print(kneipp_stats)

Warning message:
"There were 8 warnings in `dplyr::summarise()`.
The first warning was:
i In argument: `min_CCQ = min(followup_antibiotics_last_winter, na.rm = TRUE)`.
i In group 3: `group = "Control"`, `sex = NA`.
Caused by warning in `min()`:
! no non-missing arguments to min; returning Inf
i Run `dplyr::last_dplyr_warnings()` to see the 7 remaining warnings."
`summarise()` has grouped output by 'group'. You can override using the
`.groups` argument.


# A tibble: 10 x 7
# Groups:   group [3]
   group     sex     n mean_CCQ sd_CCQ min_CCQ max_CCQ
   <chr>   <dbl> <int>    <dbl>  <dbl>   <dbl>   <dbl>
 1 Control     1    62     1.06  0.248       1       2
 2 Control     2    49     1.06  0.242       1       2
 3 Control    NA     4   NaN    NA         Inf    -Inf
 4 Kneipp      1    52     1.29  0.457       1       2
 5 Kneipp      2    39     1.08  0.270       1       2
 6 Kneipp     NA    14   NaN    NA         Inf    -Inf
 7 Unknown     1   128     1.10  0.300       1       2
 8 Unknown     2   124     1.05  0.228       1       2
 9 Unknown     3     1   NaN    NA         Inf    -Inf
10 Unknown    NA     3   NaN    NA         Inf    -Inf


GI

In [56]:
# Model for followup_GI_last_winter by sex
GI_male_model <- lmer(followup_GI_last_winter ~ group + baseline_GI_last_winter + (1 | pairs) + (1 | kindergarten_ID),
                      data = ITT_population %>% filter(sex == "1"))

GI_female_model <- lmer(followup_GI_last_winter ~ group + baseline_GI_last_winter + (1 | pairs) + (1 | kindergarten_ID),
                        data = ITT_population %>% filter(sex == "2"))

boundary (singular) fit: see help('isSingular')

boundary (singular) fit: see help('isSingular')



In [57]:
summary(GI_male_model)
confint(GI_male_model, method = "Wald")

Linear mixed model fit by REML ['lmerMod']
Formula: followup_GI_last_winter ~ group + baseline_GI_last_winter + (1 |  
    pairs) + (1 | kindergarten_ID)
   Data: ITT_population %>% filter(sex == "1")

REML criterion at convergence: 94.2

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-0.8931 -0.6035 -0.3264 -0.3251  2.0865 

Random effects:
 Groups          Name        Variance  Std.Dev.
 kindergarten_ID (Intercept) 0.0002713 0.01647 
 pairs           (Intercept) 0.0000000 0.00000 
 Residual                    0.1717744 0.41446 
Number of obs: 81, groups:  kindergarten_ID, 9; pairs, 4

Fixed effects:
                        Estimate Std. Error t value
(Intercept)              1.02111    0.14284   7.149
groupKneipp              0.11696    0.09545   1.225
baseline_GI_last_winter  0.11486    0.10626   1.081

Correlation of Fixed Effects:
            (Intr) grpKnp
groupKneipp -0.106       
bsln_GI_ls_ -0.897 -0.208
optimizer (nloptwrap) convergence code: 0 (OK)
boundary (sing

,2.5 %,97.5 %
.sig01,NA,NA
.sig02,NA,NA
.sigma,NA,NA
(Intercept),0.74114150,1.3010693
groupKneipp,-0.07011204,0.3040385
baseline_GI_last_winter,-0.09340237,0.3231318


In [58]:
summary(GI_female_model)
confint(GI_female_model, method = "Wald")

Linear mixed model fit by REML ['lmerMod']
Formula: followup_GI_last_winter ~ group + baseline_GI_last_winter + (1 |  
    pairs) + (1 | kindergarten_ID)
   Data: ITT_population %>% filter(sex == "2")

REML criterion at convergence: 85.6

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-0.8641 -0.6787 -0.4806  1.4408  1.6852 

Random effects:
 Groups          Name        Variance Std.Dev.
 kindergarten_ID (Intercept) 0.0000   0.0000  
 pairs           (Intercept) 0.0000   0.0000  
 Residual                    0.2132   0.4617  
Number of obs: 62, groups:  kindergarten_ID, 9; pairs, 4

Fixed effects:
                        Estimate Std. Error t value
(Intercept)              1.13633    0.19379   5.864
groupKneipp              0.09145    0.11800   0.775
baseline_GI_last_winter  0.08558    0.13234   0.647

Correlation of Fixed Effects:
            (Intr) grpKnp
groupKneipp -0.388       
bsln_GI_ls_ -0.907  0.106
optimizer (nloptwrap) convergence code: 0 (OK)
boundary (singular

,2.5 %,97.5 %
.sig01,NA,NA
.sig02,NA,NA
.sigma,NA,NA
(Intercept),0.7565039,1.5161650
groupKneipp,-0.1398393,0.3227312
baseline_GI_last_winter,-0.1738047,0.3449716


In [59]:
# For Kneipp group
kneipp_stats <- ITT_population %>%
  dplyr::group_by(group, sex) %>%
  dplyr::summarise(
    n = n(),
    mean_CCQ = mean(followup_GI_last_winter, na.rm = TRUE),
    sd_CCQ = sd(followup_GI_last_winter, na.rm = TRUE),
    min_CCQ = min(followup_GI_last_winter, na.rm = TRUE),
    max_CCQ = max(followup_GI_last_winter, na.rm = TRUE)
  )

# Display the result
print(kneipp_stats)

Warning message:
"There were 8 warnings in `dplyr::summarise()`.
The first warning was:
i In argument: `min_CCQ = min(followup_GI_last_winter, na.rm = TRUE)`.
i In group 3: `group = "Control"`, `sex = NA`.
Caused by warning in `min()`:
! no non-missing arguments to min; returning Inf
i Run `dplyr::last_dplyr_warnings()` to see the 7 remaining warnings."
`summarise()` has grouped output by 'group'. You can override using the
`.groups` argument.


# A tibble: 10 x 7
# Groups:   group [3]
   group     sex     n mean_CCQ sd_CCQ min_CCQ max_CCQ
   <chr>   <dbl> <int>    <dbl>  <dbl>   <dbl>   <dbl>
 1 Control     1    62     1.18  0.385       1       2
 2 Control     2    49     1.29  0.456       1       2
 3 Control    NA     4   NaN    NA         Inf    -Inf
 4 Kneipp      1    52     1.31  0.466       1       2
 5 Kneipp      2    39     1.33  0.478       1       2
 6 Kneipp     NA    14   NaN    NA         Inf    -Inf
 7 Unknown     1   128     1.22  0.416       1       2
 8 Unknown     2   124     1.30  0.460       1       2
 9 Unknown     3     1   NaN    NA         Inf    -Inf
10 Unknown    NA     3   NaN    NA         Inf    -Inf


PP

# BASELINE CHAR ITT


In [61]:
head(ITT_population)

serial_ID,questionnaire_time_point,zipcode,number_adults,number_minors,age_oldest,age_youngest,family_status,parent1,parent2,...,baseline_bronchitis_last_winter,baseline_lung_inflammation_last_winter,baseline_antibiotics_last_winter,baseline_GI_last_winter,CCQ_followup_score,followup_cold_last_winter,followup_bronchitis_last_winter,followup_lung_inflammation_last_winter,followup_antibiotics_last_winter,followup_GI_last_winter
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,...,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
ECMQH4UTAE,V0-2,18,2,2,4,2,2,1,1,...,NaN,NaN,NaN,NaN,7.000000,2,1,1,1,2
5RGKME5182,V0-2,154,2,1,NA,NA,1,1,2,...,NaN,NaN,NaN,NaN,0.000000,2,1,1,1,1
CGW4K85NWW,V0-2,NA,2,2,6,4,1,1,2,...,NaN,NaN,NaN,NaN,7.714286,1,1,1,1,1
8V2D3P53RS,V0-2,17,2,1,NA,NA,2,1,2,...,NaN,NaN,NaN,NaN,0.000000,2,1,1,1,2
32Z14S942R,V0-2,124,1,2,13,5,3,1,NA,...,1,1,1,2,9.714286,2,1,1,1,2
SMP6X4EAFF,V0-2,128,1,1,NA,NA,3,2,1,...,NaN,NaN,NaN,NaN,9.285714,2,1,1,1,1


In [62]:
colnames(ITT_population)

[1] "serial_ID"                             
 [2] "questionnaire_time_point"              
 [3] "zipcode"                               
 [4] "number_adults"                         
 [5] "number_minors"                         
 [6] "age_oldest"                            
 [7] "age_youngest"                          
 [8] "family_status"                         
 [9] "parent1"                               
[10] "parent2"                               
[11] "education_parent1"                     
[12] "education_parent2"                     
[13] "occupation_parent1"                    
[14] "occupation_parent2"                    
[15] "household_income"                      
[16] "kindergarten_ID"                       
[17] "child_number"                          
[18] "acclimatization"                       
[19] "birth_order"                           
[20] "weight"                                
[21] "height"                                
[22] "sex"                                   
[23] "vaccination_status"                    
[24] "pre_existing_disease"                  
[25] "chronic_disease"                       
[26] "time_outside"                          
[27] "sport_and_club"                        
[28] "cold_last_winter"                      
[29] "bronchitis_last_winter"                
[30] "lung_inflammation_last_winter"         
[31] "antibiotics_last_winter"               
[32] "GI_last_winter"                        
[33] "sick_days_due_sickness"                
[34] "sick_days_due_to_other"                
[35] "CCQ_fever"                             
[36] "CCQ_chills"                            
[37] "CCQ_muscle_pain"                       
[38] "CCQ_watery_eyes"                       
[39] "CCQ_runny_nose"                        
[40] "CCQ_sneezing"                          
[41] "CCQ_sore_throat"                       
[42] "CCQ_cough"                             
[43] "CCQ_chest_pain"                        
[44] "abdominal_pain"                        
[45] "diarrhea"                              
[46] "vomiting"                              
[47] "Number_sickdays_last_winter"           
[48] "child_age_months"                      
[49] "sleep_duration_weekdays"               
[50] "sleep_duration_weekends"               
[51] "time_point_label"                      
[52] "group"                                 
[53] "pairs"                                 
[54] "CCQ_sum_score"                         
[55] "CCQ_baseline_score"                    
[56] "baseline_cold_last_winter"             
[57] "baseline_bronchitis_last_winter"       
[58] "baseline_lung_inflammation_last_winter"
[59] "baseline_antibiotics_last_winter"      
[60] "baseline_GI_last_winter"               
[61] "CCQ_followup_score"                    
[62] "followup_cold_last_winter"             
[63] "followup_bronchitis_last_winter"       
[64] "followup_lung_inflammation_last_winter"
[65] "followup_antibiotics_last_winter"      
[66] "followup_GI_last_winter"

In [63]:
# Filter for baseline time points (V0 and V0-2), and exclude groups that are not Control or Kneipp
baseline_data <- ITT_population %>%
  filter(questionnaire_time_point %in% c("V0", "V0-2") & group %in% c("Control", "Kneipp") & sex %in% c("1", "2"))

In [64]:
# Summary table for numeric values
baseline_summary_numeric <- baseline_data %>%
  dplyr::group_by(group) %>%
  dplyr::summarise(
    group = first(group),  # Ensure 'group' column is preserved
    # Sex distribution
    Male = sum(sex == 1, na.rm = TRUE),
    Female = sum(sex == 2, na.rm = TRUE),
    Other = sum(sex == 3, na.rm = TRUE),
    Total_Sex = n(),

    # Age in months
    Age_Mean = mean(child_age_months, na.rm = TRUE),
    Age_SD = sd(child_age_months, na.rm = TRUE),

    # Acclimatization to kindergarten in months
    Acclimatization_Mean = mean(acclimatization * 12, na.rm = TRUE),
    Acclimatization_SD = sd(acclimatization * 12, na.rm = TRUE),

    # Weight (kg)
    Weight_Mean = mean(weight, na.rm = TRUE),
    Weight_SD = sd(weight, na.rm = TRUE),

    # Height (cm)
    Height_Mean = mean(height, na.rm = TRUE),
    Height_SD = sd(height, na.rm = TRUE),

    # Sleep duration (hours)
    Sleep_Weekdays_Mean = mean(sleep_duration_weekdays, na.rm = TRUE),
    Sleep_Weekdays_SD = sd(sleep_duration_weekdays, na.rm = TRUE),
    
    Sleep_Weekends_Mean = mean(sleep_duration_weekends, na.rm = TRUE),
    Sleep_Weekends_SD = sd(sleep_duration_weekends, na.rm = TRUE),

    # Adults and children in household
    Adults_Mean = mean(number_adults, na.rm = TRUE),
    Adults_SD = sd(number_adults, na.rm = TRUE),

    Children_Mean = mean(number_minors, na.rm = TRUE),
    Children_SD = sd(number_minors, na.rm = TRUE),

    # Age of oldest and youngest child
    Oldest_Child_Age_Mean = mean(age_oldest, na.rm = TRUE),
    Oldest_Child_Age_SD = sd(age_oldest, na.rm = TRUE),
    
    Youngest_Child_Age_Mean = mean(age_youngest, na.rm = TRUE),
    Youngest_Child_Age_SD = sd(age_youngest, na.rm = TRUE)
  )

In [65]:
# Summary table for character values
baseline_summary_character <- baseline_data %>%
  dplyr::group_by(group) %>%
  dplyr::summarise(
    group = first(group),  # Ensure 'group' column is preserved
    # Age range (character values)
    Age_Range = paste0(range(child_age_months, na.rm = TRUE), collapse = " - "),
    
    # Acclimatization range (converting from years to months)
    Acclimatization_Range = paste0(range(acclimatization * 12, na.rm = TRUE), collapse = " - "),
    
    # Weight range
    Weight_Range = paste0(range(weight, na.rm = TRUE), collapse = " - "),
    
    # Height range
    Height_Range = paste0(range(height, na.rm = TRUE), collapse = " - "),

    # Sleep Weekdays
    Sleep_Weekdays_Range = paste0(range(sleep_duration_weekdays, na.rm = TRUE), collapse = " - "),

    Sleep_Weekends_Range = paste0(range(sleep_duration_weekends, na.rm = TRUE), collapse = " - "),

    # number of adults
    Adults_Range = paste0(range(number_adults, na.rm = TRUE), collapse = " - "),

    # number of minors
    Children_Range = paste0(range(number_minors, na.rm = TRUE), collapse = " - "),

    # oldest and youngest child
    Oldest_Child_Age_Range = paste0(range(age_oldest, na.rm = TRUE), collapse = " - "),
    Youngest_Child_Age_Range = paste0(range(age_youngest, na.rm = TRUE), collapse = " - ")
  )

In [66]:
# Summary table for categorical values (including percentages)
baseline_summary_categorical <- baseline_data %>%
  dplyr::group_by(group) %>%
  dplyr::summarise(
    group = first(group),  # Ensure 'group' column is preserved
    # Vaccination status
    All_STIKO = sum(vaccination_status == 1, na.rm = TRUE),
    Some_STIKO = sum(vaccination_status == 2, na.rm = TRUE),
    Not_Vaccinated = sum(vaccination_status == 3, na.rm = TRUE),

    # Previous illnesses
    Previous_Illness_Yes = sum(pre_existing_disease == 2, na.rm = TRUE),
    Previous_Illness_No = sum(pre_existing_disease == 1, na.rm = TRUE),

    # Chronic illnesses
    Chronic_Illness_Yes = sum(chronic_disease == 2, na.rm = TRUE),
    Chronic_Illness_No = sum(chronic_disease == 1, na.rm = TRUE),

    # Time spent outdoors
    Outdoors_30min = sum(time_outside == 1, na.rm = TRUE),
    Outdoors_30_60min = sum(time_outside == 2, na.rm = TRUE),
    Outdoors_60_90min = sum(time_outside == 3, na.rm = TRUE),
    Outdoors_90min = sum(time_outside == 4, na.rm = TRUE),

    # Attending sport/club
    Sport_Yes = sum(sport_and_club == 2, na.rm = TRUE),
    Sport_No = sum(sport_and_club == 1, na.rm = TRUE),

    # Marital status
    Married = sum(family_status == 1, na.rm = TRUE),
    Cohabiting = sum(family_status == 2, na.rm = TRUE),
    Single_Parent = sum(family_status == 3, na.rm = TRUE),
    Separated_Divorced = sum(family_status == 4, na.rm = TRUE),
    Widowed = sum(family_status == 5, na.rm = TRUE),
    Patchwork_Families = sum(family_status == 6, na.rm = TRUE),

    # Education for Parent 1
    Parent1_Secondary = sum(education_parent1 == 2, na.rm = TRUE),
    Parent1_University_Entrance = sum(education_parent1 == 3, na.rm = TRUE),
    Parent1_Vocational = sum(education_parent1 == 4, na.rm = TRUE),
    Parent1_University = sum(education_parent1 == 5, na.rm = TRUE),
    Parent1_No_Qualification = sum(education_parent1 == 1, na.rm = TRUE),

    # Education for Parent 2
    Parent2_Secondary = sum(education_parent2 == 2, na.rm = TRUE),
    Parent2_University_Entrance = sum(education_parent2 == 3, na.rm = TRUE),
    Parent2_Vocational = sum(education_parent2 == 4, na.rm = TRUE),
    Parent2_University = sum(education_parent2 == 5, na.rm = TRUE),
    Parent2_No_Qualification = sum(education_parent2 == 1, na.rm = TRUE),

    # Employment for Parent 1
    Parent1_Pupil = sum(occupation_parent1 == 1, na.rm = TRUE),
    Parent1_Training = sum(occupation_parent1 == 2, na.rm = TRUE),
    Parent1_Student = sum(occupation_parent1 == 3, na.rm = TRUE),
    Parent1_Employee = sum(occupation_parent1 == 4, na.rm = TRUE),
    Parent1_Civil_Servant = sum(occupation_parent1 == 5, na.rm = TRUE),
    Parent1_Self_Employed = sum(occupation_parent1 == 6, na.rm = TRUE),
    Parent1_Unemployed = sum(occupation_parent1 == 7, na.rm = TRUE),
    Parent1_Other = sum(occupation_parent1 == 8, na.rm = TRUE),

    # Employment for Parent 2
    Parent2_Pupil = sum(occupation_parent2 == 1, na.rm = TRUE),
    Parent2_Training = sum(occupation_parent2 == 2, na.rm = TRUE),
    Parent2_Student = sum(occupation_parent2 == 3, na.rm = TRUE),
    Parent2_Employee = sum(occupation_parent2 == 4, na.rm = TRUE),
    Parent2_Civil_Servant = sum(occupation_parent2 == 5, na.rm = TRUE),
    Parent2_Self_Employed = sum(occupation_parent2 == 6, na.rm = TRUE),
    Parent2_Unemployed = sum(occupation_parent2 == 7, na.rm = TRUE),
    Parent2_Other = sum(occupation_parent2 == 8, na.rm = TRUE),

    # Household income
    Income_Less_Than_499 = sum(household_income == 2, na.rm = TRUE),
    Income_500_999 = sum(household_income == 4, na.rm = TRUE),
    Income_1000_1499 = sum(household_income == 5, na.rm = TRUE),
    Income_1500_1999 = sum(household_income == 6, na.rm = TRUE),
    Income_2000_2499 = sum(household_income == 7, na.rm = TRUE),
    Income_2500_2999 = sum(household_income == 8, na.rm = TRUE),
    Income_3000_3499 = sum(household_income == 9, na.rm = TRUE),
    Income_3500_3999 = sum(household_income == 10, na.rm = TRUE),
    Income_4000_And_More = sum(household_income == 11, na.rm = TRUE)
  )

In [67]:
# Pivot the numeric summary data to be vertical
baseline_summary_numeric_long <- baseline_summary_numeric %>%
  tidyr::pivot_longer(cols = -group, names_to = "Variable", values_to = "Value")


In [68]:
# Pivot the character summary data to be vertical
baseline_summary_character_long <- baseline_summary_character %>%
  tidyr::pivot_longer(cols = -group, names_to = "Variable", values_to = "Value")


In [69]:
# Pivot the categorical summary data to be vertical
baseline_summary_categorical_long <- baseline_summary_categorical %>%
  tidyr::pivot_longer(cols = -group, names_to = "Variable", values_to = "Value")


In [70]:
# Print the numeric summary
cat("Numeric Summary:\n")
print(baseline_summary_numeric_long, n = Inf)

Numeric Summary:
# A tibble: 48 x 3
   group   Variable                  Value
   <chr>   <chr>                     <dbl>
 1 Control Male                     62    
 2 Control Female                   49    
 3 Control Other                     0    
 4 Control Total_Sex               111    
 5 Control Age_Mean                 53.4  
 6 Control Age_SD                   15.4  
 7 Control Acclimatization_Mean     24.5  
 8 Control Acclimatization_SD       16.0  
 9 Control Weight_Mean              17.8  
10 Control Weight_SD                 3.89 
11 Control Height_Mean             107.   
12 Control Height_SD                12.2  
13 Control Sleep_Weekdays_Mean      12.5  
14 Control Sleep_Weekdays_SD         3.27 
15 Control Sleep_Weekends_Mean      11.9  
16 Control Sleep_Weekends_SD         4.42 
17 Control Adults_Mean               1.95 
18 Control Adults_SD                 0.341
19 Control Children_Mean             1.85 
20 Control Children_SD               0.753
21 Control Oldest_

In [71]:
# Print the character summary
cat("\nCharacter Summary:\n")
print(baseline_summary_character_long, n = Inf)


Character Summary:
# A tibble: 20 x 3
   group   Variable                 Value                              
   <chr>   <chr>                    <chr>                              
 1 Control Age_Range                14.6976748430428 - 80.2516790772376
 2 Control Acclimatization_Range    12 - 72                            
 3 Control Weight_Range             11 - 30                            
 4 Control Height_Range             63 - 130                           
 5 Control Sleep_Weekdays_Range     -13.6666666666667 - 15             
 6 Control Sleep_Weekends_Range     -12 - 15                           
 7 Control Adults_Range             1 - 3                              
 8 Control Children_Range           1 - 4                              
 9 Control Oldest_Child_Age_Range   2 - 17                             
10 Control Youngest_Child_Age_Range 0 - 7                              
11 Kneipp  Age_Range                21.5055345670901 - 81.7114952912834
12 Kneipp  Acclimatizatio

In [72]:
# Print the categorical summary
cat("\nCategorical Summary:\n")
print(baseline_summary_categorical_long, n = Inf)


Categorical Summary:
# A tibble: 108 x 3
    group   Variable                    Value
    <chr>   <chr>                       <int>
  1 Control All_STIKO                     101
  2 Control Some_STIKO                     10
  3 Control Not_Vaccinated                  0
  4 Control Previous_Illness_Yes           11
  5 Control Previous_Illness_No           100
  6 Control Chronic_Illness_Yes             6
  7 Control Chronic_Illness_No            105
  8 Control Outdoors_30min                  2
  9 Control Outdoors_30_60min              27
 10 Control Outdoors_60_90min              41
 11 Control Outdoors_90min                 41
 12 Control Sport_Yes                      40
 13 Control Sport_No                       71
 14 Control Married                        68
 15 Control Cohabiting                     32
 16 Control Single_Parent                   7
 17 Control Separated_Divorced              3
 18 Control Widowed                         1
 19 Control Patchwork_Families        